In [2]:
import kagglehub
import os
import shutil
from ultralytics import YOLO
import cv2
import numpy as np
from deepface import DeepFace

# Definir función para cargar PNGs
def load_image(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    return img

def overlay_image(bg, overlay, x, y, scale=1.0):
    if overlay is None:
        return bg

    h, w = overlay.shape[:2]
    new_w, new_h = int(w * scale), int(h * scale)
    overlay = cv2.resize(overlay, (new_w, new_h))

    orig_x, orig_y = x, y

    overlay_x1 = max(0, -orig_x)
    overlay_y1 = max(0, -orig_y)

    x = max(0, orig_x)
    y = max(0, orig_y)

    overlay_visible_w = min(new_w - overlay_x1, bg.shape[1] - x)
    overlay_visible_h = min(new_h - overlay_y1, bg.shape[0] - y)

    if overlay_visible_w <= 0 or overlay_visible_h <= 0:
        return bg 

    overlay_crop = overlay[overlay_y1:overlay_y1+overlay_visible_h,
                           overlay_x1:overlay_x1+overlay_visible_w]

    if overlay.shape[2] == 4:
        alpha = overlay_crop[:, :, 3] / 255.0
        for c in range(3):
            bg[y:y+overlay_visible_h, x:x+overlay_visible_w, c] = \
                alpha * overlay_crop[:, :, c] + \
                (1 - alpha) * bg[y:y+overlay_visible_h, x:x+overlay_visible_w, c]
    else:
        bg[y:y+overlay_visible_h, x:x+overlay_visible_w] = overlay_crop

    return bg


c:\Users\Usuario\anaconda3\envs\VC_P5\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# Descargar dataset FER (emociones)
dataset_path = kagglehub.dataset_download("ananthu017/emotion-detection-fer")

base = dataset_path

# Crear el modelo YOLOv8 pequeño para clasificación
model = YOLO("yolov8s-cls.pt")

model.train(
    data=base,
    epochs=12,
    imgsz=128,
    batch=32
)

In [6]:
# Cargar modelo entrenado
model = YOLO("runs/classify/train/weights/best.pt")

# Detector de rostros
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


smile = load_image("assets/smile.png")
cloud = load_image("assets/rain.png")
smoke = load_image("assets/smoke.png")
sweat = load_image("assets/sweat.png")


cap = cv2.VideoCapture(0)

# Bucle de procesamiento de video
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640, 480))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (224, 224))
        result = model(face_resized, verbose=False)[0]
        emotion_id = result.probs.top1
        emotion = result.names[emotion_id]

        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

         # Efectivos según emoción
        if emotion == "happy" and smile is not None:
            scale_factor = w / 4000 
            pos_x = x + w//2 - int(smile.shape[1] * scale_factor / 2)
            pos_y = y - int(smile.shape[0] * scale_factor) - 10
            frame = overlay_image(frame, smile, pos_x, pos_y, scale=scale_factor)

        elif emotion == "sad" and cloud is not None:
            scale_factor = w / 600
            pos_x = x + w//2 - int(cloud.shape[1] * scale_factor / 2)
            pos_y = y - int(cloud.shape[0] * scale_factor) - 20
            frame = overlay_image(frame, cloud, pos_x, pos_y, scale=scale_factor)

        elif emotion == "angry" and smoke is not None:
            scale_factor = w / 2500
            flipped_smoke = cv2.flip(smoke, 1)
            ear_height = y + int(h * 0.05)
            frame = overlay_image(frame, flipped_smoke, x - int(30 * scale_factor)-150, ear_height, scale=scale_factor)
            frame = overlay_image(frame, smoke, x + w - int(30 * scale_factor), ear_height, scale=scale_factor)

        elif emotion == "surprised":
            cv2.putText(frame, "!", (x + w//2, y - 40), cv2.FONT_HERSHEY_SIMPLEX, 3, (0,0,255), 6)

        elif emotion == "fearful" and sweat is not None:
            scale_factor = w / 6000
            frame = overlay_image(frame, sweat, x + int(w*0.1), y + int(h*0.1), scale=scale_factor)
            frame = overlay_image(frame, sweat, x + int(w*0.55), y + int(h*0.1), scale=scale_factor)

        elif emotion == "neutral":
            overlay = frame.copy()
            overlay[:] = (100, 100, 100)
            frame = cv2.addWeighted(overlay, 0.25, frame, 0.75, 0)

    cv2.imshow("Emociones", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
top_hat = load_image("assets/top_hat.png")
monocle = load_image("assets/monocle.png")
mustache = load_image("assets/mustache.png")


# Filtro de estática 
def add_static(frame, amount=0.12):
    noise = np.random.randint(0, 255, (frame.shape[0], frame.shape[1]), dtype=np.uint8)
    noise = cv2.cvtColor(noise, cv2.COLOR_GRAY2BGR)
    return cv2.addWeighted(frame, 1 - amount, noise, amount, 0)



# Bucle de procesamiento de video
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640, 480))
 
    # Deepface
    detections = DeepFace.extract_faces(
        img_path=frame,
        detector_backend='opencv', 
        enforce_detection=False
    )

    if len(detections) > 0:
        face_obj = detections[0]

        fa = face_obj["facial_area"]
        x = int(fa["x"])
        y = int(fa["y"])
        w = int(fa["w"])
        h = int(fa["h"])


        # Sombrero de copa
        hat_scale = w / 200
        hat_x = x + w//2 - int(top_hat.shape[1] * hat_scale / 2)
        hat_y = y - int(top_hat.shape[0] * hat_scale) +10
        frame = overlay_image(frame, top_hat, hat_x, hat_y, hat_scale)

        # Monóculo
        if monocle is not None:
            mono_scale = w / 2000
            eye_x = x + int(w * 0.68)
            eye_y = y + int(h * 0.52)

            mono_x = eye_x - int(monocle.shape[1] * mono_scale / 2)
            mono_y = eye_y - int(monocle.shape[0] * mono_scale / 2)

            frame = overlay_image(frame, monocle, mono_x, mono_y, mono_scale)

        # Bigote
        if mustache is not None:
            must_scale = w / 1500

            must_y = y + int(h * 0.33)
            must_x = x + w//2 - int(mustache.shape[1] * must_scale / 2)

            frame = overlay_image(frame, mustache, must_x, must_y, must_scale)


    # Fitro blanco y negro
    old_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    old_frame = cv2.cvtColor(old_frame, cv2.COLOR_GRAY2BGR)

    # Añadir estática
    old_frame = add_static(old_frame, amount=0.15)

    cv2.imshow("Hombre Monopoly", old_frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
